In [24]:
# 라이브러리 호출
import pandas as pd

# 경고창 제거
import warnings
warnings.filterwarnings('ignore')

# 데이터 불러오기
df = pd.read_csv('../../../../dataset/clean_feature/model_df.csv',index_col=0)

# feature, target 분리
X = df.drop(columns='VKOSPI_Label', axis=1)
y = df[['VKOSPI_Label']]

## 1. Wrapper

In [25]:
# Forward Selection
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# 학습 데이터와 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Forward feature selection 수행
selected_features = []
best_score = 0

while len(selected_features) < X.shape[1]:
    best_feature = None
    best_model = None
    best_score_local = 0

    for feature in X_train.columns:
        if feature not in selected_features:
            features = selected_features + [feature]
            X_train_selected = X_train[features]
            X_val_selected = X_val[features]

            model = DecisionTreeClassifier()
            model.fit(X_train_selected, y_train)
            score = model.score(X_val_selected, y_val)

            if score > best_score_local:
                best_score_local = score
                best_feature = feature
                best_model = model

    if best_score_local > best_score:
        selected_features.append(best_feature)
        best_score = best_score_local
        print(f"Selected feature: {best_feature}, Accuracy: {best_score:.4f}")

    else:
        break

print("\nForward selected features:")
Forward = selected_features


Selected feature: put_volume_fluc, Accuracy: 0.9091

Forward selected features:


In [26]:
# Backward Elimination
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split


# 학습 데이터와 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Backward feature selection 수행
selected_features = X_train.columns.tolist()
best_score = 0

while len(selected_features) > 0:
    worst_feature = None
    best_model = None
    best_score_local = 0

    for feature in selected_features:
        features = selected_features.copy()
        features.remove(feature)

        X_train_selected = X_train[features]
        X_val_selected = X_val[features]

        model = DecisionTreeClassifier()
        model.fit(X_train_selected, y_train)
        score = model.score(X_val_selected, y_val)

        if score > best_score_local:
            best_score_local = score
            worst_feature = feature
            best_model = model

    if best_score_local > best_score:
        selected_features.remove(worst_feature)
        best_score = best_score_local
        print(f"Removed feature: {worst_feature}, Accuracy: {best_score:.4f}")

    else:
        break

print("\nFinal selected features:")
Backward = selected_features


Removed feature: basis, Accuracy: 0.8182

Final selected features:


In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
# 학습 데이터와 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Stepwise feature selection 수행을 위한 리스트와 초기 점수 설정
selected_features = []
best_score = 0

# 순방향 선택 (Forward step)
while len(selected_features) < X.shape[1]:
    best_feature = None
    best_model = None
    best_score_local = 0

    for feature in X_train.columns:
        if feature not in selected_features:
            features = selected_features + [feature]
            X_train_selected = X_train[features]
            X_val_selected = X_val[features]

            # DecisionTree 모델 사용
            model = DecisionTreeClassifier()
            model.fit(X_train_selected, y_train)
            score = model.score(X_val_selected, y_val)

            if score > best_score_local:
                best_score_local = score
                best_feature = feature
                best_model = model

    if best_score_local > best_score:
        selected_features.append(best_feature)
        best_score = best_score_local
        print(f"선택된 피처: {best_feature}, 정확도: {best_score:.4f}")

    else:
        break

# 역방향 제거 (Backward step)
while len(selected_features) > 0:
    worst_feature = None
    best_model = None
    best_score_local = 0

    for feature in selected_features:
        features = selected_features.copy()
        features.remove(feature)

        X_train_selected = X_train[features]
        X_val_selected = X_val[features]

        # DecisionTree 모델 사용
        model = DecisionTreeClassifier()
        model.fit(X_train_selected, y_train)
        score = model.score(X_val_selected, y_val)

        if score > best_score_local:
            best_score_local = score
            worst_feature = feature
            best_model = model

    if best_score_local > best_score:
        selected_features.remove(worst_feature)
        best_score = best_score_local
        print(f"제거된 피처: {worst_feature}, 정확도: {best_score:.4f}")

    else:
        break

print("\n최종 선택된 피처:")
Stepwise= selected_features
Stepwise


선택된 피처: put_volume_fluc, 정확도: 0.9091
선택된 피처: basis_disparate_ratio, 정확도: 1.0000

최종 선택된 피처:


['put_volume_fluc', 'basis_disparate_ratio']

## 2. Filter

In [28]:
# H가 1인 경우 이분산성 / H가 0인 경우 등분산
from scipy.stats import bartlett
def bartlett_test(col, p_value = 0.05, H = 1):
    list= []
    for i in col:
        T, p_val =bartlett(df[df['VKOSPI_Label']==1][i], df[df['VKOSPI_Label']==0][i]) 
        list.append([i, p_val])

    list = pd.DataFrame(list, columns = ['변수', 'p_value'])
    if H == 1:
        a = list[(list['p_value'] < p_value)][['변수', 'p_value']].sort_values('p_value')
        return a
    else:
        a = list[(list['p_value'] >= p_value)][['변수', 'p_value']].sort_values('p_value')
        return a

In [29]:
# 이분산성 변수
x_hetero = bartlett_test(df.columns, H = 1)
# 등분산성 변수
x_homo = bartlett_test(df.columns, H = 0)

In [30]:
# t-test

import scipy.stats as stats
def t_test(col, col_h0, col_h1, p_value = 0.05):
    list= []
    for i in col:
        if (col_h0['변수']==i).any():
            t_stat, p_val = stats.ttest_ind(df[df['VKOSPI_Label']==1][i], df[df['VKOSPI_Label']==0][i], equal_var=True) # 등분산성 : wald t-test
            list.append([i, p_val])
        elif (col_h1['변수']==i).any():
            t_stat, p_val = stats.ttest_ind(df[df['VKOSPI_Label']==1][i], df[df['VKOSPI_Label']==0][i], equal_var=False) # 이분산성 : welch’s t-test
            list.append([i, p_val])

    list = pd.DataFrame(list, columns = ['변수', 'p_value'])
    a = list[(list['p_value'] < p_value)][['변수', 'p_value']].sort_values('p_value')
    return a

In [31]:
# 2) t_test 결과 p_value < 0.05보다 작은 유의한 변수 가져오기
x_ttest = t_test(df.columns, x_homo, x_hetero, p_value=0.1)
print("유의한 피쳐 수 :", len(x_ttest))
x_ttest.sort_values(by="변수", ascending=True)

유의한 피쳐 수 : 9


,변수,p_value
0,basis,0.040543
2,basis_disparate_ratio,0.035870
1,basis_risk,0.039087
19,call_volume_fluc,0.037327
9,kospi_p_change(%),0.005819
10,kospi_p_range,0.031252
11,ks200_p_change(%),0.001677
18,put_volume_fluc,0.009576
25,vix_close_change,0.027702


In [32]:
ttest = list(x_ttest['변수'])
ttest

['ks200_p_change(%)',
 'kospi_p_change(%)',
 'put_volume_fluc',
 'vix_close_change',
 'kospi_p_range',
 'basis_disparate_ratio',
 'call_volume_fluc',
 'basis_risk',
 'basis']

## 3. Embedded

In [33]:
# DecisionTree 모델 및 하이퍼파라미터 그리드 설정
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV


dt_model = DecisionTreeClassifier()
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV를 이용한 하이퍼파라미터 최적화
grid_search = GridSearchCV(dt_model, param_grid=param_grid, return_train_score=True, cv=5)
grid_search.fit(X_train, y_train)

# 결과를 데이터프레임으로 변환 및 정렬
df = pd.DataFrame(grid_search.cv_results_)
df = df.sort_values(by=['rank_test_score'], ascending=True)

# 주요 컬럼만 선택하여 출력
result_df = df[['params', 'mean_train_score', 'mean_test_score', 'rank_test_score']]

# 최적의 하이퍼파라미터와 그 때의 점수 출력
best_params = grid_search.best_params_
best_score = grid_search.best_score_

result_df, best_params, best_score


(                                               params  mean_train_score  \
 45  {'criterion': 'entropy', 'max_depth': None, 'm...          1.000000   
 86  {'criterion': 'entropy', 'max_depth': 40, 'min...          0.918992   
 68  {'criterion': 'entropy', 'max_depth': 20, 'min...          0.918992   
 64  {'criterion': 'entropy', 'max_depth': 20, 'min...          0.988571   
 77  {'criterion': 'entropy', 'max_depth': 30, 'min...          0.918992   
 ..                                                ...               ...   
 40  {'criterion': 'gini', 'max_depth': 40, 'min_sa...          0.935966   
 53  {'criterion': 'entropy', 'max_depth': None, 'm...          0.883866   
 70  {'criterion': 'entropy', 'max_depth': 20, 'min...          0.895630   
 22  {'criterion': 'gini', 'max_depth': 20, 'min_sa...          0.935966   
 30  {'criterion': 'gini', 'max_depth': 30, 'min_sa...          0.947731   
 
     mean_test_score  rank_test_score  
 45         0.747222                1  
 86   

In [34]:
# 최적의 하이퍼파라미터로 DecisionTree 모델 학습
dt_best = DecisionTreeClassifier(criterion= 'entropy',max_depth= 20,min_samples_leaf= 2,min_samples_split= 2).fit(X_train, y_train)

# 피처 중요도를 데이터프레임으로 변환
df_embedded = pd.DataFrame()
df_embedded['feature'] = X_train.columns
df_embedded['importance'] = dt_best.feature_importances_

# 중요도가 0인 피처 제거
df_embedded = df_embedded[df_embedded['importance'] != 0]

df_embedded


,feature,importance
6,cny_night_change,0.069066
9,kospi_p_change(%),0.460493
11,ks200_p_change(%),0.196046
18,put_volume_fluc,0.150045
22,sp_day_fluc_range,0.124349


In [35]:
# 임베디드에서 선택된 피처
Embedded = df_embedded['feature'].values.tolist()
print('df_embedded에서 선택된 피처 수 {0:1.0f}'.format(len(df_embedded)), '개')
Embedded

df_embedded에서 선택된 피처 수 5 개


['cny_night_change',
 'kospi_p_change(%)',
 'ks200_p_change(%)',
 'put_volume_fluc',
 'sp_day_fluc_range']

In [38]:
list_Forward = list(Forward)
list_Backward = list(Backward)
list_Stepwise = list(Stepwise)
list_ttest= list(ttest)
list_Embedded = list(Embedded)
list_col_all = X_train.columns

In [40]:
def func_Forward(x):
    if x in list_Forward:
        return 1
    else:
        return 0
    
def func_Backward(x):
    if x in list_Backward:
        return 1
    else:
        return 0


def func_Stepwise(x):
    if x in list_Stepwise:
        return 1
    else:
        return 0

def func_ttest(x):
    if x in list_ttest:
        return 1
    else:
        return 0
    
def func_Embedded(x):
    if x in list_Embedded:
        return 1
    else:
        return 0

In [41]:
# 2번 선택된 Feature

feature_counts = pd.DataFrame()
feature_counts['Feature'] = list_col_all
feature_counts['Forward'] = list_col_all.map(func_Forward)
feature_counts['Backward'] = list_col_all.map(func_Backward)
feature_counts['Stepwise'] = list_col_all.map(func_Stepwise)
feature_counts['ttest'] = list_col_all.map(func_ttest)
feature_counts['Embedded'] = list_col_all.map(func_Embedded)

feature_counts["total"] = feature_counts["Forward"]+feature_counts['Backward']+feature_counts["Stepwise"]+feature_counts["ttest"]+feature_counts["Embedded"]
feature_final = feature_counts[feature_counts["total"]>=2]
list_feature_final = list(feature_final["Feature"])
print("선택된 피쳐수 :", len(list_feature_final))
feature_final

선택된 피쳐수 : 10


,Feature,Forward,Backward,Stepwise,ttest,Embedded,total
1,basis_risk,0,1,0,1,0,2
2,basis_disparate_ratio,0,1,1,1,0,3
6,cny_night_change,0,1,0,0,1,2
9,kospi_p_change(%),0,1,0,1,1,3
10,kospi_p_range,0,1,0,1,0,2
11,ks200_p_change(%),0,1,0,1,1,3
18,put_volume_fluc,1,1,1,1,1,5
19,call_volume_fluc,0,1,0,1,0,2
22,sp_day_fluc_range,0,1,0,0,1,2
25,vix_close_change,0,1,0,1,0,2


In [42]:
# 3번 선택된 Feature

feature_counts = pd.DataFrame()
feature_counts['Feature'] = list_col_all
feature_counts['Forward'] = list_col_all.map(func_Forward)
feature_counts['Backward'] = list_col_all.map(func_Backward)
feature_counts['Stepwise'] = list_col_all.map(func_Stepwise)
feature_counts['ttest'] = list_col_all.map(func_ttest)
feature_counts['Embedded'] = list_col_all.map(func_Embedded)

feature_counts["total"] = feature_counts["Forward"]+feature_counts['Backward']+feature_counts["Stepwise"]+feature_counts["ttest"]+feature_counts["Embedded"]
feature_final = feature_counts[feature_counts["total"]>=3]
list_feature_final = list(feature_final["Feature"])
print("선택된 피쳐수 :", len(list_feature_final))
feature_final

선택된 피쳐수 : 4


,Feature,Forward,Backward,Stepwise,ttest,Embedded,total
2,basis_disparate_ratio,0,1,1,1,0,3
9,kospi_p_change(%),0,1,0,1,1,3
11,ks200_p_change(%),0,1,0,1,1,3
18,put_volume_fluc,1,1,1,1,1,5
